# Model Testing

| Date | User | Change Type | Remarks |  
| ---- | ---- | ----------- | ------- |
| 12/06/2025   | Martin | Create  | Baseline model with Decision Tree and HistGradientBoosting regressor | 

__Scores__

- Baseline (DecisionTree | Full Data): 

# Content

* [Baseline Model](#baseline-model)

# Baseline Model

Baseline model serves as the starting point for all future scores. Using a Decision Tree with KFold Cross Validation

In [1]:
import polars as pl
import numpy as np

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import HistGradientBoostingRegressor
from scipy.stats import pearsonr

In [2]:
df = pl.read_csv("data/clean/orig_plus.csv")

# Remove timestamp
df = df.drop('timestamp')

In [3]:
df.head()

bid_qty,ask_qty,buy_qty,sell_qty,volume,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,X11,X12,X13,X14,X15,X16,X17,X18,X19,X20,X21,X22,X23,X24,X25,X26,X27,X28,X29,X30,X31,X32,…,X863,X864,X865,X866,X867,X868,X869,X870,X871,X872,X873,X874,X875,X876,X877,X878,X879,X880,X881,X882,X883,X884,X885,X886,X887,X888,X889,X890,label,bair,moi,ofr,lpr,obs,vwoi,va,obrsi
f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
15.283,8.425,176.405,44.984,221.389,0.121263,-0.41769,0.005399,0.125948,0.058359,0.027359,0.03578,0.068219,1.034825,-0.029575,0.327805,0.485823,0.668596,0.617389,0.770037,0.857631,1.754456,0.572503,0.883229,0.58567,0.816321,0.529973,0.508244,0.448616,1.341892,1.406392,0.953631,1.183991,1.474789,0.774389,0.660586,0.269043,…,0.21857,0.0,1.728155,0.62414,0.0,-0.051211,0.0,0.0,0.0,0.0,0.691754,0.242124,2.096157,3.369195,0.244667,0.286611,0.722679,0.901931,1.000007,1.925423,1.847943,0.005676,0.190791,0.369691,0.37763,0.210153,0.159183,0.530636,0.562539,1.814006,3.921505,0.59362,3.589058,138.279,0.59362,626.407,57.874677
38.59,2.336,525.846,321.95,847.796,0.302841,-0.049576,0.356667,0.481087,0.237954,0.208359,0.217057,0.249624,0.948694,-0.183488,0.150526,0.308421,0.492232,0.529787,0.682958,0.770965,1.686504,0.273357,0.591695,0.442391,0.674792,0.460741,0.439681,0.380399,1.304113,1.003783,0.776628,1.015943,1.312735,0.696895,0.584217,0.231104,…,0.088014,0.0,1.665698,0.622775,0.0,-0.079621,0.0,0.0,0.0,0.0,0.691665,0.242091,2.46103,4.127584,0.321394,0.31246,0.746452,0.912371,1.003153,1.928569,1.849468,0.005227,0.18466,0.363642,0.374515,0.209573,0.158963,0.530269,0.533686,16.519692,1.633316,0.240501,1.74055,240.15,0.240501,626.407,57.874677
0.442,60.25,159.227,136.369,295.596,0.167462,-0.291212,0.083138,0.206881,0.101727,0.072778,0.081564,0.114166,0.896459,-0.261779,0.044571,0.200608,0.384558,0.476229,0.629848,0.718232,1.656707,0.140156,0.457268,0.376524,0.610116,0.429751,0.409316,0.350359,1.28325,0.760801,0.670816,0.917205,1.219124,0.653355,0.541739,0.210095,…,-0.147363,0.0,1.666893,0.621414,0.0,-0.080427,0.0,0.0,0.0,0.0,0.691674,0.242093,2.493249,4.182112,0.326701,0.314636,0.746681,0.911129,1.002502,1.928047,1.849282,0.004796,0.178719,0.357689,0.371424,0.208993,0.158744,0.529901,0.546505,0.007336,1.167619,0.077329,0.812073,-36.95,0.077329,-552.2,57.874677
4.865,21.016,335.742,124.963,460.705,0.072944,-0.43659,-0.102483,0.017551,0.007149,-0.021681,-0.012936,0.019634,0.732634,-0.535845,-0.273947,-0.124959,0.056438,0.311539,0.465377,0.554022,1.663491,0.152084,0.468778,0.383696,0.618529,0.435326,0.415523,0.356895,1.319538,0.955549,0.789646,1.044941,1.353001,0.72392,0.613462,0.246212,…,-0.09459,0.0,1.735322,0.620057,0.0,-0.094702,0.0,0.0,0.0,0.0,0.69121,0.24193,2.525526,4.292975,0.350791,0.32357,0.753829,0.913363,1.002985,1.928621,1.849608,0.004398,0.172967,0.351832,0.368358,0.208416,0.158524,0.529534,0.357703,0.23149,2.686731,0.457514,2.33326,194.628,0.457514,165.109,57.874677
27.158,3.451,98.411,44.407,142.818,0.17382,-0.213489,0.096067,0.215709,0.107133,0.078976,0.087818,0.120426,0.763537,-0.430945,-0.205298,-0.062118,0.117266,0.341493,0.495591,0.584519,1.668419,0.156177,0.472732,0.3871,0.623192,0.439034,0.419868,0.361572,1.324595,0.90546,0.78375,1.047708,1.36188,0.732001,0.622712,0.251095,…,0.162221,0.0,1.712096,0.618703,0.0,-0.091884,0.0,0.0,0.0,0.0,0.691207,0.241928,2.52443,4.306694,0.335599,0.31907,0.747533,0.908904,1.001286,1.927084,1.84895,0.004008,0.167391,0.346066,0.365314,0.207839,0.158304,0.529167,0.362452,7.869603,2.216115,0.378132,2.623783,77.711,0.378132,-317.887,57.874677


In [4]:
# Split into X and y
y = df['label'].to_numpy()
X = df.drop('label').to_numpy()

# Standardise the data
scaler = StandardScaler()
X = scaler.fit_transform(X)

X.shape, y.shape

((525887, 882), (525887,))

In [5]:
N_SPLIT = 5

# Create KFold cross validation and train
kf = KFold(n_splits=N_SPLIT)

scores = []
for i, (train_idx, test_idx) in enumerate(kf.split(X)):
  print(f"Fold {i+1}")

  train_X, train_y = X[train_idx], y[train_idx]
  test_X, test_y = X[test_idx], y[test_idx]

  # model = DecisionTreeRegressor(
  #   max_depth=6,
  #   random_state=43
  # )
  model = HistGradientBoostingRegressor()
  model.fit(train_X, train_y)

  results = model.predict(test_X)
  score = pearsonr(results, test_y).statistic

  scores.append(score)

  print(f"Pearson Coefficient Score: {score}")
  print("---------------------------")


Fold 1
Pearson Coefficient Score: 0.14227886093064868
---------------------------
Fold 2
Pearson Coefficient Score: 0.10016111447317327
---------------------------
Fold 3
Pearson Coefficient Score: 0.03382046192678599
---------------------------
Fold 4
Pearson Coefficient Score: 0.095336635473158
---------------------------
Fold 5
Pearson Coefficient Score: 0.06157752412538714
---------------------------
